In [16]:
text = (
       'Hello, how are you? I am Romeo.\n'
       'Hello, Romeo My name is Juliet. Nice to meet you.\n'
       'Nice meet you too. How are you today?\n'
       'Great. My baseball team won the competition.\n'
       'Oh Congratulations, Juliet\n'
       'Thanks you Romeo'
   )

In [17]:
import re
import random
from hoho_bert import *

In [18]:
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')
word_list = list(set(' '.join(sentences).split()))

In [19]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
    number_dict = {i: w for i, w in enumerate(word_dict)}
    vocab_size = len(word_dict)

In [20]:
word_dict

{'[PAD]': 0,
 '[CLS]': 1,
 '[SEP]': 2,
 '[MASK]': 3,
 'to': 4,
 'thanks': 5,
 'great': 6,
 'you': 7,
 'is': 8,
 'meet': 9,
 'name': 10,
 'competition': 11,
 'baseball': 12,
 'i': 13,
 'hello': 14,
 'juliet': 15,
 'congratulations': 16,
 'today': 17,
 'am': 18,
 'the': 19,
 'how': 20,
 'romeo': 21,
 'oh': 22,
 'my': 23,
 'team': 24,
 'nice': 25,
 'won': 26,
 'are': 27,
 'too': 28}

In [22]:
# TEST 1
print(get_attn_pad_mask(input_ids, input_ids)[0][0], input_ids[0])

NameError: name 'input_ids' is not defined

In [23]:
# TEST 2
emb = Embedding()
embeds = emb(input_ids, segment_ids)
attenM = get_attn_pad_mask(input_ids, input_ids)
SDPA = ScaledDotProductAttention()(embeds, embeds, embeds, attenM)

S, C, A= SDPA

print('Masks:', masks[0][0])
print()
print('Scores: ', S[0][0])
print('Attention Scores after softmax: ', A[0][0])

NameError: name 'vocab_size' is not defined

In [ ]:
# TEST 3

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

model = BERT()
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(100):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens)
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext)
    loss = loss_lm + loss_clsf
    
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()
    
# Predict mask tokens
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)